### Section 1: Sudoku Solver using Integer Programming

#### Problem Statement:

The Sudoku puzzle is a classic 9x9 grid-based puzzle where the goal is to fill each cell with a number from 1 to 9 such that:

1. Each **row** contains all numbers from 1 to 9 exactly once.
2. Each **column** contains all numbers from 1 to 9 exactly once.
3. Each **3x3 sub-grid** (blocks within the main 9x9 grid) also contains all numbers from 1 to 9 exactly once.

Mathematically, this problem can be represented using a set of constraints over a binary variable `x[i, j, k]`, where `i` and `j` represent the cell position in the grid, and `k` represents a candidate number. If `x[i, j, k] = 1`, it means that the cell `(i, j)` contains the number `k`.

##### Constraint Definitions:

- **Single Number per Cell**: Ensure that each cell contains exactly one number.
  
  $$ \sum_{k=1}^{9} x[i, j, k] = 1, \, \forall \, i, j $$

- **Row and Column Constraints**: Each number must appear exactly once in each row and each column.

  $$ \sum_{j=1}^{9} x[i, j, k] = 1, \, \forall \, i, k $$
  
  $$ \sum_{i=1}^{9} x[i, j, k] = 1, \, \forall \, j, k $$

- **Sub-grid Constraints**: Each number must appear exactly once in each 3x3 sub-grid.

  $$ \sum_{i', j' \in \, \text{block}(i, j)} x[i', j', k] = 1, \, \forall \, k $$

##### Objective Function:

For the standard Sudoku problem, we are primarily interested in finding a feasible solution that satisfies all the constraints. Therefore, the objective function can be set to a constant.

However, in this **Sudoku** variation, the objective is to maximize the sum of elements on the main diagonal of the grid. This adds an extra layer of complexity, requiring custom constraints to prioritize certain cells.

#### Implementation:

The following code implements a Sudoku solver in Julia using the `JuMP` library and the `HiGHS` solver. The solver not only satisfies the Sudoku constraints but also solves a modified version of the puzzle, where the sum of the diagonal elements is maximized.

In [1]:
# Import the necessary packages
using JuMP, HiGHS

# Define the problem size (standard 9x9 Sudoku)
n = 9

# Initialize the model with the HiGHS solver
sudoku = Model(HiGHS.Optimizer)

# Define binary variables for Sudoku
@variable(sudoku, x[1:n, 1:n, 1:n], Bin)

# Constraint: Only one number per cell
for i in 1:n, j in 1:n
    @constraint(sudoku, sum(x[i, j, k] for k in 1:n) == 1)
end

# Constraint: Each number must appear exactly once per row and column
for k in 1:n
    for idx in 1:n
        @constraint(sudoku, sum(x[idx, j, k] for j in 1:n) == 1)
        @constraint(sudoku, sum(x[i, idx, k] for i in 1:n) == 1)
    end
end

# Constraint: Each number must appear exactly once in each block
for block_i in 0:2
    for block_j in 0:2
        for k in 1:n
            @constraint(sudoku, sum(x[3*block_i + i, 3*block_j + j, k] for i in 1:3, j in 1:3) == 1)
        end
    end
end

# Objective: Maximize the sum of the elements on the main diagonal (Jeff-Doku variation)
@objective(sudoku, Max, sum(k * x[i, i, k] for i in 1:n, k in 1:n))

# Solve the problem
optimize!(sudoku)

# Check if the model has a valid solution
if termination_status(sudoku) == MOI.OPTIMAL
    # Extract and print the solution
    solution = zeros(Int, n, n)
    for i in 1:n
        for j in 1:n
            for k in 1:n
                if value(x[i, j, k]) > 0.5  # Assuming binary variables are either 0 or 1
                    solution[i, j] = k
                end
            end
        end
    end
    println("Solution to Sudoku:")
    println(solution)
else
    println("No optimal solution found!")
end

Running HiGHS 1.7.0 (git hash: 50670fd4c): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+00, 9e+00]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+00]
Presolving model
324 rows, 729 cols, 2916 nonzeros  0s
324 rows, 729 cols, 2916 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   324 rows
   729 cols (729 binary, 0 integer, 0 implied int., 0 continuous)
   2916 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   135             -inf                 inf        0      0      0         0     0.0s
         0       0         0   0.00%   72              -inf                 inf        0      0      8       603     0.0s
 R       0       0         0   0.00%   72    

### Section 2: Grandmother Puzzle Solver Using Advanced Constraint Modeling

#### Problem Statement:

In this puzzle, there are five grandmothers, each with a daughter, a son-in-law, and a grandson. We need to determine the correct assignments for these relationships based on a set of complex rules and constraints:

1. Each **grandmother** has exactly one **daughter**, one **son-in-law**, and one **grandson**.
2. There are five grandmothers: Maxine, Mabel, Mavis, Millie, and Martha.
3. The daughters’ names are: Carla, Carol, Cindy, Cathy, and Caren.
4. The sons-in-law are: John, Jake, Jack, Joe, and Jason.
5. The grandsons are: Tom, Tex, Tim, Tip, and Tab.
6. A set of clues and rules determines which daughter, son-in-law, and grandson each grandmother has. 

##### **Clues and Constraints:**

1. **Maxine’s daughter** was not Carla.
  
   $$ xD[\text{Maxine}, \text{Carla}] = 0 $$

2. **Mavis’s son-in-law** was not Jack.
  
   $$ xH[\text{Mavis}, \text{Jack}] = 0 $$

3. **Tim’s mother** was Cindy, who was married to Jake.

   $$ xD[g, \text{Cindy}] + xH[g, \text{Jake}] + xS[g, \text{Tim}] = 3 \cdot z[g], \, \forall \, g $$

4. **Mabel, Millie, and Martha** did not have daughters named Carla or Carol.

   $$ xD[g, \text{Carla}] = 0, \, xD[g, \text{Carol}] = 0, \, \forall \, g \in \{ \text{Mabel}, \text{Millie}, \text{Martha} \} $$

5. **Martha’s son-in-law** was not John, because John was married to Caren, and their son was named Tom.

   $$ xD[\text{Martha}, \text{Caren}] = 0 $$

6. **Millie’s son-in-law** was either Joe or Jason, and her grandson was either Tip or Tab.

   $$ xH[\text{Millie}, \text{Joe}] + xH[\text{Millie}, \text{Jason}] = 1 $$

7. **Tab** did not have a grandmother named Mavis.

   $$ xS[\text{Mavis}, \text{Tab}] = 0 $$

##### **Mathematical Model:**

The problem is modeled using the following binary variables:

- `xD[g, d]`: 1 if grandmother `g` has daughter `d`, 0 otherwise.
- `xH[g, h]`: 1 if grandmother `g` has son-in-law `h`, 0 otherwise.
- `xS[g, s]`: 1 if grandmother `g` has grandson `s`, 0 otherwise.

##### **Objective Function:**

This is a **feasibility problem**, so there is no optimization objective. We only need to find a feasible assignment that satisfies all constraints.

The following code implements the Grandmother Puzzle Solver using JuMP and Gurobi in Julia.

In [5]:
using JuMP, Gurobi

# Create a new model with Gurobi optimizer
m = Model(Gurobi.Optimizer)

# Define sets using symbols for clarity and indexing
G = [:Maxine, :Mabel, :Mavis, :Millie, :Martha]
D = [:Carla, :Carol, :Cindy, :Cathy, :Caren]
H = [:John, :Jake, :Jack, :Joe, :Jason]
S = [:Tom, :Tex, :Tim, :Tip, :Tab]

# Define variables: 1 if relationship holds, 0 otherwise
@variable(m, xD[G,D], Bin)  # Daughter relationships
@variable(m, xH[G,H], Bin)  # Son-in-law relationships
@variable(m, xS[G,S], Bin)  # Grandson relationships
@variable(m, z[G], Bin)     # Auxiliary variable to activate relationship set
@variable(m, y[G], Bin)     # Auxiliary variable to activate relationship set

# Constraint: Each person has exactly one role in each grandmother's story
for g in G
    @constraint(m, sum(xD[g, d] for d in D) == 1)
    @constraint(m, sum(xH[g, h] for h in H) == 1)
    @constraint(m, sum(xS[g, s] for s in S) == 1)
end

# Constraint: Each role must be uniquely filled across all grandmothers
for d in D
    @constraint(m, sum(xD[g, d] for g in G) == 1)
end
for h in H
    @constraint(m, sum(xH[g, h] for g in G) == 1)
end
for s in S
    @constraint(m, sum(xS[g, s] for g in G) == 1)
end

# Additional Provided Constraints

# Maxine’s Daughter was not Carla
@constraint(m, xD[:Maxine, :Carla] == 0)

# Mavis’ son-in-law was not named Jack, and Cathy was married to Joe, but their son was not named Tab
@constraint(m, xH[:Mavis, :Jack] == 0)
for g in G
    @constraint(m, xD[g, :Cathy] + xH[g, :Joe] <= 1 + (1 - xS[g, :Tab]))
end

# Tim’s mother was not named either Carol or Carla, because Tim’s mother was Jake’s wife Cindy

for g in G
    @constraint(m, xD[g, :Carol] + xS[g, :Tim] <= 1)
    @constraint(m, xD[g, :Carla] + xS[g, :Tim] <= 1)
end

for g in G
    @constraint(m, xD[g, :Cindy] + xH[g, :Jake] + xS[g, :Tim] >= 3 * z[g])
    @constraint(m, xD[g, :Cindy] <= z[g])
    @constraint(m, xH[g, :Jake] <= z[g])
    @constraint(m, xS[g, :Tim] <= z[g])
end

# Mabel, Millie, and Martha did not have daughters named Carla or Carol, and Martha’s son-in-law was not John because John was married to Caren, and their son was named Tom
for g in [:Mabel, :Millie, :Martha], d in [:Carla, :Carol]
    @constraint(m, xD[g, d] == 0)
end

@constraint(m, xH[:Martha, :John] == 0)

for g in G
    @constraint(m, xD[g, :Caren] + xH[g, :John] + xS[g, :Tom] >= 3 * y[g])
    @constraint(m, xD[g, :Caren] <= y[g])
    @constraint(m, xH[g, :John] <= y[g])
    @constraint(m, xS[g, :Tom] <= y[g])
end

# Millie’s son-in-law was named either Joe or Jason, and her grandson was named either Tip or Tab
@constraint(m, xH[:Millie, :Joe] + xH[:Millie, :Jason] == 1)
@constraint(m, xS[:Millie, :Tip] + xS[:Millie, :Tab] == 1)

# Tab did not have a grandmother named Mavis
@constraint(m, xS[:Mavis, :Tab] == 0)

# Solve the model
optimize!(m)

# Check the solution status and print the results if feasible
if termination_status(m) == MOI.OPTIMAL
    println("Optimal solution found.")
    for g in G
        daughter = first([d for d in D if value(xD[g, d]) > 0.5])
        son_in_law = first([h for h in H if value(xH[g, h]) > 0.5])
        grandson = first([s for s in S if value(xS[g, s]) > 0.5])
        println("Grandma ", g, " has daughter ", daughter, ", son-in-law ", son_in_law, ", and grandson ", grandson)
    end
else
    println("No feasible solution found. Model is infeasible.")
    computeIIS(m)
    write(m, "model.ilp")  # Save the infeasibility report if needed
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-23
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 7800X3D 8-Core Processor, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 97 rows, 85 columns and 299 nonzeros
Model fingerprint: 0x5cc3c26d
Variable types: 0 continuous, 85 integer (85 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 97 rows and 85 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, g